In [1]:
%pip install pandas
%pip install matplotlib
%pip install seaborn


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
PATH_TO_EVALUATION_RESULT_ITERATION_1_QUANT = '../Ergebnisse/evaluation_iteration1_quantitativ.csv'

In [11]:
def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"No file found in path: '{file_path}'")
        return None
    except Exception as e:
        print(f"Error reading data: {str(e)}")
        return None

df_iteration_1 = read_csv_to_dataframe(PATH_TO_EVALUATION_RESULT_ITERATION_1_QUANT)

if df_iteration_1 is not None:
    print(df_iteration_1.head())

  PROBAND_CODE        ENTWURF  FRAGE_CODE ITEM_CODE  \
0   PROBAND_01  FARBCODIERUNG  F_SKALA_01   ITEM_01   
1   PROBAND_01  FARBCODIERUNG  F_SKALA_02   ITEM_02   
2   PROBAND_01  FARBCODIERUNG  F_SKALA_03   ITEM_03   
3   PROBAND_01  FARBCODIERUNG  F_SKALA_04   ITEM_04   
4   PROBAND_01  FARBCODIERUNG  F_SKALA_05   ITEM_05   

                                          FRAGE_TEXT ITEM_REFERENZ  \
0  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
1  Die Anwendung der Farbcodierung mit Legende zu...     Anwendung   
2  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
3  Die Darstellung von konkreten Aufrufbeziehunge...   Darstellung   
4  Das Gesamtbild erscheint mir durch die Farbcod...    Gesamtbild   

       ITEM_NEGATIV      ITEM_POSITIV       UX_DIMENSION  ANTWORT INFO  
0    unverständlich      verständlich  DURCHSCHAUBARKEIT        4  NaN  
1  schwer zu lernen  leicht zu lernen  DURCHSCHAUBARKEIT        5  NaN  
2       kompliziert          

In [68]:
import pandas as pd

def calculate_min_max_mode_by_dim(df):
    # Group by 'ENTWURF' and then 'UX_DIMENSION'
    grouped_data = df.groupby(['ENTWURF', 'UX_DIMENSION'])

    # Calculate min, max, and mode for each group
    result = grouped_data.agg({
        'ANTWORT': ['min', 'max', lambda x: x.mode().iloc[0] if not x.mode().empty else None]
    })

    # Rename the columns for clarity
    result.columns = ['Minimum', 'Maximum', 'Modus']

    # Create classes and calculate occurrences for each class
    bins = [0, 2, 3, 5, 7]
    class_occurrences = grouped_data['ANTWORT'].apply(lambda x: pd.cut(x, bins=bins, labels=False, right=False).value_counts()).unstack(fill_value=0)

    # Add the occurrences columns to the result
    result = pd.concat([result, class_occurrences], axis=1)

    # Rename the columns based on the class intervals
    result.columns = ['Minimum', 'Maximum', 'Modus', '1-2', '3', '4-5', '6-7']

    # Sort the index based on the specified order of 'ENTWURF'
    result = result.loc[['FARBCODIERUNG', 'PFEILSYSTEM', 'ANIMATION']]

    return result

result_df = calculate_min_max_mode_by_dim(df_iteration_1)
print(result_df)

                                 Minimum  Maximum  Modus  1-2  3  4-5  6-7
ENTWURF       UX_DIMENSION                                                
FARBCODIERUNG DURCHSCHAUBARKEIT        3        7      6    0  0    4   11
              EFFIZIENZ                2        7      5    0  1    4   12
PFEILSYSTEM   DURCHSCHAUBARKEIT        3        7      6    0  0    6   10
              EFFIZIENZ                3        7      6    0  0    6    9
ANIMATION     DURCHSCHAUBARKEIT        2        7      6    0  2    2   10
              EFFIZIENZ                1        7      5    1  2    6    8


In [77]:
def plot_mode_min_max(data_frame):
    plt.figure(figsize=(6, 5.5))

    # Define a custom color palette
    custom_palette = {'DURCHSCHAUBARKEIT': '#2979b5', 'EFFIZIENZ': '#4da6ff'}

    # Plot the bar plot without dotting and hatching
    ax = sns.barplot(x="ENTWURF", y="Modus", hue="UX_DIMENSION", data=result_df, errorbar=None, width=0.5, palette=custom_palette)

    # Add error bars for Min and Max values with a different color
    min_values = result_df['Minimum']
    max_values = result_df['Maximum']
    for i, (bar, min_val, max_val) in enumerate(zip(ax.patches, min_values, max_values)):
        hue_index = i % len(result_df.index.levels[1])
        ax.errorbar(
            x=bar.get_x() + bar.get_width() / 2,
            y=result_df['Modus'].iloc[i],
            yerr=[[result_df['Modus'].iloc[i] - min_val], [max_val - result_df['Modus'].iloc[i]]],
            color='black',  # Change the color to red (you can choose any color you prefer)
            fmt='-',  # Use a solid line
            capsize=5,
        )

    # Set labels and legend
    ax.set_title(None)  # Remove the title
    plt.xlabel('ENTWURF')
    plt.ylabel('Modus')

    # Modify legend labels for side-by-side display
    legend_labels = result_df.index.levels[1].tolist()
    plt.legend(title='UX_DIMENSION', labels=legend_labels, loc='upper right')

    # Set y-axis limits with a lower limit of 0
    plt.ylim(0, 9)

    # Hide specific y-axis tick labels
    plt.yticks([1, 2, 3, 4, 5, 6, 7])

    plt.xlabel('Entwurf')  # Change the x-axis label
    plt.ylabel('Modus, Min, Max')  # Change the y-axis label

    # Modify legend labels for side-by-side display
    legend_labels = result_df.index.levels[1].tolist()
    plt.legend(title='UX Dimension', labels=legend_labels, loc='upper right')

    # Show the plot
    plt.show()


SyntaxError: expected ':' (3520231222.py, line 1)